In [1]:
import sqlalchemy
import pyodbc
from urllib.parse import quote
import pandas as pd
from tqdm import tqdm
engine=sqlalchemy.create_engine('mssql+pyodbc://SAL_USR:%s@172.16.20.36:1433/HDM_DB?driver=ODBC+Driver+17+for+SQL+Server' % quote('Sal@read1'))

In [2]:
mobiles = '7452887135|9504324572|9894974346|6353343865|7200803720|9413801830|9814017119|9811974783|9099916594|9899189300'
mobiles_to_search = mobiles.split('|')

In [3]:
mobiles_to_search

['7452887135',
 '9504324572',
 '9894974346',
 '6353343865',
 '7200803720',
 '9413801830',
 '9814017119',
 '9811974783',
 '9099916594',
 '9899189300']

In [4]:
import os

In [5]:
for mob in tqdm(mobiles_to_search):
    sql = f"with cte1 as(select distinct pan,batchid,rptsernum,CONCAT_WS('|',batchid,rptsernum) brn,ACCOUNTNUM,PERSONNAME,COMMADDRESS,'ARFINP' as source,mobile,TELEPHONE from [HDM_DB].[FINCORE_BACKUP_CC].[arfinp] where TELEPHONE like '%{mob}%' or MOBILE like '%{mob}%' and batchid>2104010000\
           union\
           select distinct pan,batchid,rptsernum,CONCAT_WS('|',batchid,rptsernum) brn,ACCOUNTNUM,PERSONNAME,COMMADDRESS,'ARFLPE' as source,mobile,TELEPHONE from [HDM_DB].[FINCORE_BACKUP_CC].[arfinp] where TELEPHONE like '%{mob}%' or MOBILE like '%{mob}%' and batchid>2104010000\
           union\
           select distinct pan,batchid,rptsernum,CONCAT_WS('|',batchid,rptsernum) brn,'null' AS ACCOUNTNUM,CUSTOMERNAME,ADDRESS,'TRFTRN' as source,mobile,TELEPHONE from [HDM_DB].[FINCORE_BACKUP_CC].[trftrn] where TELEPHONE like '%{mob}%' or MOBILE like '%{mob}%' and batchid>2104010000)\
            select batchid,rptsernum,CONCAT_WS('|',batchid,rptsernum) brn,CONCAT_WS('|',GROUNDSOFSUSP,DETAILSOFINVESTIGATIONS) gosdoi,MAINPERSONNAME,SOURCEOFALERT from [FINCORE_BACKUP_CC].[REPORT] (nolock) where CONCAT_WS('|',batchid,rptsernum) in (select brn from cte1)"
    result = pd.read_sql_query(sql,engine)
    result.to_csv(os.path.join(r'C:\Users\SAL005\Desktop\adhoc request','mobiles_search_report.csv'), index = False, mode = 'a', header = not os.path.join(r'C:\Users\SAL005\Desktop\adhoc request','mobiles_search_report.csv')) 

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [47:45<00:00, 286.55s/it]
